<a href="https://colab.research.google.com/github/jkim1134/repository-bioinformatics/blob/main/Fly_manure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3

"""Set up Qiime 2 on Google colab.

Do not use this on o local machine, especially not as an admin!
"""

import os
import sys
import shutil
from subprocess import Popen, PIPE, run

r = Popen(["pip", "install", "rich"])
r.wait()
from rich.console import Console  # noqa
con = Console()

PREFIX = "/usr/local/miniforge3/"

has_conda = "conda version" in os.popen("%s/bin/conda info" % PREFIX).read()
qiime_installed = os.path.exists(os.path.join(PREFIX, "envs", "qiime2", "bin", "qiime"))
qiime_active = "QIIME 2 release:" in os.popen("qiime info").read()


MINICONDA_PATH = (
    "https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh"
)

QIIME_YAML_TEMPLATE = (
    "https://data.qiime2.org/distro/amplicon/qiime2-amplicon-{version}-py{python}-linux-conda.yml"
)

if len(sys.argv) == 2:
    version = sys.argv[1]
else:
    version = "2025.4"

qiime_ver = tuple(int(v) for v in version.split("."))

if qiime_ver < (2021, 4):
    pyver = "36"
elif qiime_ver < (2024, 5):
    pyver = "38"
elif qiime_ver < (2024, 10):
    pyver = "39"
else:
  pyver = "310"

CONDA = "mamba"
CONDA_ARGS = ["-q"] if CONDA == "conda" else ["-y"]

if tuple(float(v) for v in version.split(".")) < (2023, 9):
    QIIME_YAML_TEMPLATE = (
        "https://data.qiime2.org/distro/core/qiime2-{version}-py{python}-linux-conda.yml"
    )

QIIME_YAML_URL = QIIME_YAML_TEMPLATE.format(version=version, python=pyver)
QIIME_YAML = os.path.basename(QIIME_YAML_URL)


def cleanup():
    """Remove downloaded files."""
    if os.path.exists(os.path.basename(MINICONDA_PATH)):
        os.remove(os.path.basename(MINICONDA_PATH))
    if os.path.exists(QIIME_YAML):
        os.remove(QIIME_YAML)
    if os.path.exists("/content/sample_data"):
        shutil.rmtree("/content/sample_data")
    con.log(":broom: Cleaned up unneeded files.")


def run_and_check(args, check, message, failure, success, console=con):
    """Run a command and check that it worked."""
    console.log(message)
    r = Popen(args, env=os.environ, stdout=PIPE, stderr=PIPE,
              universal_newlines=True)
    o, e = r.communicate()
    out = o + e
    if r.returncode == 0 and check in out:
        console.log("[blue]%s[/blue]" % success)
    else:
        console.log("[red]%s[/red]" % failure, out)
        open("logs.txt", "w").write(out)
        cleanup()
        sys.exit(1)

def run_in_env(cmd, env, console=con):
    """Activate a conda environment in colab."""
    conda_profile = os.path.join(PREFIX, "etc", "profile.d", "conda.sh")
    console.log(f":snake: Activating the {env} environment.")

    full = f". {conda_profile} && conda activate {env} && {cmd}"
    return run(
        full,
        shell=True,
        executable="/bin/bash",
        capture_output=True,
        text=True
    )

def mock_qiime2(console=con):
    con.log(":penguin: Setting up the Qiime2 command...")
    conda_profile = os.path.join(PREFIX, "etc", "profile.d", "conda.sh")
    with open("/usr/local/bin/qiime", "w") as mocky:
        mocky.write("#!/usr/bin/env bash")
        mocky.write(f'\n\n. {conda_profile} && conda activate qiime2 && qiime "$@"\n')
    run("chmod +x /usr/local/bin/qiime", shell=True, executable="/bin/bash")
    con.log(":penguin: Done.")

if __name__ == "__main__":
    if not has_conda:
        run_and_check(
            ["wget", MINICONDA_PATH],
            "saved",
            ":snake: Downloading miniforge...",
            "failed downloading miniforge :sob:",
            ":snake: Done."
        )

        run_and_check(
            ["bash", os.path.basename(MINICONDA_PATH), "-bfp", PREFIX],
            "installation finished.",
            ":snake: Installing miniforge...",
            "could not install miniforge :sob:",
            ":snake: Installed miniforge to `/usr/local`."
        )
    else:
        con.log(":snake: Miniforge is already installed. Skipped.")

    if not qiime_installed:
        run_and_check(
            ["wget", QIIME_YAML_URL],
            "saved",
            ":mag: Downloading Qiime 2 package list...",
            "could not download package list :sob:",
            ":mag: Done."
        )

        if CONDA == "mamba":
            CONDA_ARGS.append("-y")

        run_and_check(
            [PREFIX + "bin/" + CONDA, "env", "create", *CONDA_ARGS, "-n", "qiime2", "--file", QIIME_YAML],
            "Verifying transaction: ...working... done" if CONDA == "conda" else "Transaction finished",
            f":mag: Installing Qiime 2 ({version}). This may take a little bit.\n :clock1:",
            "could not install Qiime 2 :sob:",
            ":mag: Done."
        )

        mock_qiime2()

        con.log(":evergreen_tree: Installing empress...")
        rc = run_in_env(
            "pip install --verbose Cython && pip install iow==1.0.7 empress",
            "qiime2"
        )
        if rc.returncode == 0:
            con.log(":evergreen_tree: Done.")
        else:
            con.log("could not install Empress :sob:")
    else:
        con.log(":mag: Qiime 2 is already installed. Skipped.")
        if not qiime_active:
            mock_qiime2()

    run_and_check(
        ["qiime", "info"],
        "QIIME 2 release:",
        ":bar_chart: Checking that Qiime 2 command line works...",
        "Qiime 2 command line does not seem to work :sob:",
        ":bar_chart: Qiime 2 command line looks good :tada:"
    )

    cleanup()

    con.log("[green]Everything is A-OK. "
            "You can start using Qiime 2 now :thumbs_up:[/green]")

[21:17:28] 🐍 Downloading miniforge...                                               ]8;id=997698;file:///tmp/ipython-input-1108756784.py\ipython-input-1108756784.py]8;;\:]8;id=750163;file:///tmp/ipython-input-1108756784.py#74\74]8;;\

[21:17:29] 🐍 Done.                                                                  ]8;id=355781;file:///tmp/ipython-input-1108756784.py\ipython-input-1108756784.py]8;;\:]8;id=349549;file:///tmp/ipython-input-1108756784.py#80\80]8;;\

           🐍 Installing miniforge...                                                ]8;id=686517;file:///tmp/ipython-input-1108756784.py\ipython-input-1108756784.py]8;;\:]8;id=745357;file:///tmp/ipython-input-1108756784.py#74\74]8;;\

[21:17:42] 🐍 Installed miniforge to `/usr/local`.                                   ]8;id=610442;file:///tmp/ipython-input-1108756784.py\ipython-input-1108756784.py]8;;\:]8;id=195967;file:///tmp/ipython-input-1108756784.py#80\80]8;;\

           🔍 Downloading Qiime 2 package list...                                    ]8;id=167167;file:///tmp/ipython-input-1108756784.py\ipython-input-1108756784.py]8;;\:]8;id=243927;file:///tmp/ipython-input-1108756784.py#74\74]8;;\

...

[21:26:07] 🌲 Done.                                                                 ]8;id=716155;file:///tmp/ipython-input-1108756784.py\ipython-input-1108756784.py]8;;\:]8;id=153362;file:///tmp/ipython-input-1108756784.py#158\158]8;;\

           📊 Checking that Qiime 2 command line works...                            ]8;id=233849;file:///tmp/ipython-input-1108756784.py\ipython-input-1108756784.py]8;;\:]8;id=157733;file:///tmp/ipython-input-1108756784.py#74\74]8;;\

[21:26:55] 📊 Qiime 2 command line looks good 🎉                                     ]8;id=205214;file:///tmp/ipython-input-1108756784.py\ipython-input-1108756784.py]8;;\:]8;id=173346;file:///tmp/ipython-input-1108756784.py#80\80]8;;\

           🧹 Cleaned up unneeded files.                                             ]8;id=951841;file:///tmp/ipython-input-1108756784.py\ipython-input-1108756784.py]8;;\:]8;id=114937;file:///tmp/ipython-input-1108756784.py#69\69]8;;\

           Everything is A-OK. You can start using Qiime 2 now 👍                   ]8;id=646989;file:///tmp/ipython-input-1108756784.py\ipython-input-1108756784.py]8;;\:]8;id=201273;file:///tmp/ipython-input-1108756784.py#176\176]8;;\

In [ ]:
import os
import shutil

os.makedirs('/content/data', exist_ok=True)
os.makedirs('/content/data/classifier', exist_ok=True)

# Moving files to the right location

In [ ]:
for filename in os.listdir('/content'):
    filepath = f'/content/{filename}'
    if os.path.isfile(filepath):
        if filename.endswith('.fastq'):
            shutil.move(filepath, f'/content/data/{filename}')
        elif filename.endswith('.qza'):
            shutil.move(filepath, f'/content/data/classifier/{filename}')
        elif filename == 'true_manifest.tsv':
            shutil.move(filepath, '/content/data/manifest.tsv')
        elif filename == 'true_metadata.tsv':
            shutil.move(filepath, '/content/data/metadata.tsv')


# Fix manifest paths: change /data/ to /content/data/

In [ ]:
with open('/content/data/manifest.tsv', 'r') as f:
    content = f.read()
content = content.replace('/data/', '/content/data/')
with open('/content/data/manifest.tsv', 'w') as f:
    f.write(content)

print("Files organized!")
print("\n--- /content/data/ ---")
!ls -la /content/data/
print("\n--- /content/data/classifier/ ---")
!ls -la /content/data/classifier/

Files organized!

--- /content/data/ ---
total 1679764
drwxr-xr-x 3 root root      4096 Nov 22 21:29 .
drwxr-xr-x 1 root root      4096 Nov 22 21:29 ..
drwxr-xr-x 2 root root      4096 Nov 22 21:29 classifier
-rw-r--r-- 1 root root      1078 Nov 22 21:30 manifest.tsv
-rw-r--r-- 1 root root       567 Nov 22 21:23 metadata.tsv
-rw-r--r-- 1 root root  99008852 Nov 22 21:19 SRR25781641.fastq
-rw-r--r-- 1 root root  81976652 Nov 22 21:19 SRR25781671.fastq
-rw-r--r-- 1 root root  84670222 Nov 22 21:19 SRR25781682.fastq
-rw-r--r-- 1 root root  68081268 Nov 22 21:19 SRR25781707.fastq
-rw-r--r-- 1 root root  62215856 Nov 22 21:19 SRR25781709.fastq
-rw-r--r-- 1 root root  40244982 Nov 22 21:20 SRR25781740.fastq
-rw-r--r-- 1 root root  98195844 Nov 22 21:21 SRR25781756.fastq
-rw-r--r-- 1 root root  61696604 Nov 22 21:20 SRR25781760.fastq
-rw-r--r-- 1 root root  98879988 Nov 22 21:21 SRR25781792.fastq
-rw-r--r-- 1 root root  64754792 Nov 22 21:21 SRR25781796.fastq
-rw-r--r-- 1 root root  76991900 

# Import FASTQ files to QZA

In [ ]:
import os
import subprocess

# PREFIX is defined in the QIIME 2 setup cell (olCt0fdpPqdZ)
# For robustness, we can re-declare it or ensure the setup cell is run.
PREFIX = "/usr/local/miniforge3/"

def run_qiime(command_str):
    """Executes a QIIME 2 command within the activated qiime2 conda environment."""
    conda_profile = os.path.join(PREFIX, "etc", "profile.d", "conda.sh")
    full_command = f". {conda_profile} && conda activate qiime2 && {command_str}"
    print(f"Running QIIME 2 command:\n{command_str}\n")
    result = subprocess.run(
        full_command,
        shell=True,
        executable="/bin/bash",
        capture_output=True,
        text=True
    )
    if result.returncode != 0:
        print("Error executing QIIME 2 command:")
        print(result.stderr)
        raise Exception(f"QIIME 2 command failed with exit code {result.returncode}")
    else:
        print(result.stdout)
    return result

run_qiime('''
qiime tools import \
  --type 'SampleData[SequencesWithQuality]' \
  --input-path /content/data/manifest.tsv \
  --output-path /content/data/single_end_demux.qza \
  --input-format SingleEndFastqManifestPhred33V2
''')

Running QIIME 2 command:

qiime tools import   --type 'SampleData[SequencesWithQuality]'   --input-path /content/data/manifest.tsv   --output-path /content/data/single_end_demux.qza   --input-format SingleEndFastqManifestPhred33V2


Imported /content/data/manifest.tsv as SingleEndFastqManifestPhred33V2 to /content/data/single_end_demux.qza



CompletedProcess(args=". /usr/local/miniforge3/etc/profile.d/conda.sh && conda activate qiime2 && \nqiime tools import   --type 'SampleData[SequencesWithQuality]'   --input-path /content/data/manifest.tsv   --output-path /content/data/single_end_demux.qza   --input-format SingleEndFastqManifestPhred33V2\n", returncode=0, stdout='Imported /content/data/manifest.tsv as SingleEndFastqManifestPhred33V2 to /content/data/single_end_demux.qza\n', stderr='')

# Summarize demux for quality assessment

In [ ]:
run_qiime('''
qiime demux summarize \
  --i-data /content/data/single_end_demux.qza \
  --o-visualization /content/data/single_end_demux.qzv
''')

Running QIIME 2 command:

qiime demux summarize   --i-data /content/data/single_end_demux.qza   --o-visualization /content/data/single_end_demux.qzv


Saved Visualization to: /content/data/single_end_demux.qzv



CompletedProcess(args='. /usr/local/miniforge3/etc/profile.d/conda.sh && conda activate qiime2 && \nqiime demux summarize   --i-data /content/data/single_end_demux.qza   --o-visualization /content/data/single_end_demux.qzv\n', returncode=0, stdout='Saved Visualization to: /content/data/single_end_demux.qzv\n', stderr='')

# DADA2 denoising (adjust p-trunc-len based on quality plot if needed)

In [ ]:
run_qiime('''
qiime dada2 denoise-single \
  --i-demultiplexed-seqs /content/data/single_end_demux.qza \
  --p-trim-left 0 \
  --p-trunc-len 250 \
  --p-n-threads 6 \
  --o-representative-sequences /content/data/rep-seqs.qza \
  --o-table /content/data/feature-table.qza \
  --o-denoising-stats /content/data/denoising-stats.qza
''')

Running QIIME 2 command:

qiime dada2 denoise-single   --i-demultiplexed-seqs /content/data/single_end_demux.qza   --p-trim-left 0   --p-trunc-len 250   --p-n-threads 6   --o-representative-sequences /content/data/rep-seqs.qza   --o-table /content/data/feature-table.qza   --o-denoising-stats /content/data/denoising-stats.qza


Saved FeatureTable[Frequency] to: /content/data/feature-table.qza
Saved FeatureData[Sequence] to: /content/data/rep-seqs.qza
Saved SampleData[DADA2Stats] to: /content/data/denoising-stats.qza



CompletedProcess(args='. /usr/local/miniforge3/etc/profile.d/conda.sh && conda activate qiime2 && \nqiime dada2 denoise-single   --i-demultiplexed-seqs /content/data/single_end_demux.qza   --p-trim-left 0   --p-trunc-len 250   --p-n-threads 6   --o-representative-sequences /content/data/rep-seqs.qza   --o-table /content/data/feature-table.qza   --o-denoising-stats /content/data/denoising-stats.qza\n', returncode=0, stdout='Saved FeatureTable[Frequency] to: /content/data/feature-table.qza\nSaved FeatureData[Sequence] to: /content/data/rep-seqs.qza\nSaved SampleData[DADA2Stats] to: /content/data/denoising-stats.qza\n', stderr='')

# Generate summaries

In [ ]:
# Generate summaries
run_qiime('''
qiime feature-table summarize \
  --i-table /content/data/feature-table.qza \
  --m-sample-metadata-file /content/data/metadata.tsv \
  --o-visualization /content/data/feature-table.qzv
''')

run_qiime('''
qiime feature-table tabulate-seqs \
  --i-data /content/data/rep-seqs.qza \
  --o-visualization /content/data/rep-seqs.qzv
''')

run_qiime('''
qiime metadata tabulate \
  --m-input-file /content/data/denoising-stats.qza \
  --o-visualization /content/data/denoising-stats.qzv
''')

Running QIIME 2 command:

qiime feature-table summarize   --i-table /content/data/feature-table.qza   --m-sample-metadata-file /content/data/metadata.tsv   --o-visualization /content/data/feature-table.qzv


Saved Visualization to: /content/data/feature-table.qzv

Running QIIME 2 command:

qiime feature-table tabulate-seqs   --i-data /content/data/rep-seqs.qza   --o-visualization /content/data/rep-seqs.qzv


Saved Visualization to: /content/data/rep-seqs.qzv

Running QIIME 2 command:

qiime metadata tabulate   --m-input-file /content/data/denoising-stats.qza   --o-visualization /content/data/denoising-stats.qzv


Saved Visualization to: /content/data/denoising-stats.qzv



CompletedProcess(args='. /usr/local/miniforge3/etc/profile.d/conda.sh && conda activate qiime2 && \nqiime metadata tabulate   --m-input-file /content/data/denoising-stats.qza   --o-visualization /content/data/denoising-stats.qzv\n', returncode=0, stdout='Saved Visualization to: /content/data/denoising-stats.qzv\n', stderr='')

# ## 4. Taxonomy Classification

In [ ]:

# Classify sequences using SILVA
run_qiime('''
qiime feature-classifier classify-sklearn \
  --i-classifier /content/data/classifier/silva-138-99-nb-classifier.qza \
  --i-reads /content/data/rep-seqs.qza \
  --o-classification /content/data/taxonomy.qza
''')


Running QIIME 2 command:

qiime feature-classifier classify-sklearn   --i-classifier /content/data/classifier/silva-138-99-nb-classifier.qza   --i-reads /content/data/rep-seqs.qza   --o-classification /content/data/taxonomy.qza


Saved FeatureData[Taxonomy] to: /content/data/taxonomy.qza



CompletedProcess(args='. /usr/local/miniforge3/etc/profile.d/conda.sh && conda activate qiime2 && \nqiime feature-classifier classify-sklearn   --i-classifier /content/data/classifier/silva-138-99-nb-classifier.qza   --i-reads /content/data/rep-seqs.qza   --o-classification /content/data/taxonomy.qza\n', returncode=0, stdout='Saved FeatureData[Taxonomy] to: /content/data/taxonomy.qza\n', stderr='')

In [ ]:
# Visualize taxonomy
run_qiime('''
qiime metadata tabulate \
  --m-input-file /content/data/taxonomy.qza \
  --o-visualization /content/data/taxonomy.qzv
''')

Running QIIME 2 command:

qiime metadata tabulate   --m-input-file /content/data/taxonomy.qza   --o-visualization /content/data/taxonomy.qzv


Saved Visualization to: /content/data/taxonomy.qzv



CompletedProcess(args='. /usr/local/miniforge3/etc/profile.d/conda.sh && conda activate qiime2 && \nqiime metadata tabulate   --m-input-file /content/data/taxonomy.qza   --o-visualization /content/data/taxonomy.qzv\n', returncode=0, stdout='Saved Visualization to: /content/data/taxonomy.qzv\n', stderr='')

In [ ]:
# Create taxa barplot
run_qiime('''
qiime taxa barplot \
  --i-table /content/data/feature-table.qza \
  --i-taxonomy /content/data/taxonomy.qza \
  --m-metadata-file /content/data/metadata.tsv \
  --o-visualization /content/data/taxa-bar-plots.qzv
''')

Running QIIME 2 command:

qiime taxa barplot   --i-table /content/data/feature-table.qza   --i-taxonomy /content/data/taxonomy.qza   --m-metadata-file /content/data/metadata.tsv   --o-visualization /content/data/taxa-bar-plots.qzv


Saved Visualization to: /content/data/taxa-bar-plots.qzv



CompletedProcess(args='. /usr/local/miniforge3/etc/profile.d/conda.sh && conda activate qiime2 && \nqiime taxa barplot   --i-table /content/data/feature-table.qza   --i-taxonomy /content/data/taxonomy.qza   --m-metadata-file /content/data/metadata.tsv   --o-visualization /content/data/taxa-bar-plots.qzv\n', returncode=0, stdout='Saved Visualization to: /content/data/taxa-bar-plots.qzv\n', stderr='')

# ## 5. Phylogenetic Tree and Diversity Analysis

In [ ]:
# %% [markdown]
# ## 5. Phylogenetic Tree and Diversity Analysis

# %%
# Build phylogenetic tree
run_qiime('''
qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences /content/data/rep-seqs.qza \
  --o-alignment /content/data/aligned-rep-seqs.qza \
  --o-masked-alignment /content/data/masked-aligned-rep-seqs.qza \
  --o-tree /content/data/unrooted-tree.qza \
  --o-rooted-tree /content/data/rooted-tree.qza \
  --p-n-threads 2
''')

Running QIIME 2 command:

qiime phylogeny align-to-tree-mafft-fasttree   --i-sequences /content/data/rep-seqs.qza   --o-alignment /content/data/aligned-rep-seqs.qza   --o-masked-alignment /content/data/masked-aligned-rep-seqs.qza   --o-tree /content/data/unrooted-tree.qza   --o-rooted-tree /content/data/rooted-tree.qza   --p-n-threads 2


Saved FeatureData[AlignedSequence] to: /content/data/aligned-rep-seqs.qza
Saved FeatureData[AlignedSequence] to: /content/data/masked-aligned-rep-seqs.qza
Saved Phylogeny[Unrooted] to: /content/data/unrooted-tree.qza
Saved Phylogeny[Rooted] to: /content/data/rooted-tree.qza



CompletedProcess(args='. /usr/local/miniforge3/etc/profile.d/conda.sh && conda activate qiime2 && \nqiime phylogeny align-to-tree-mafft-fasttree   --i-sequences /content/data/rep-seqs.qza   --o-alignment /content/data/aligned-rep-seqs.qza   --o-masked-alignment /content/data/masked-aligned-rep-seqs.qza   --o-tree /content/data/unrooted-tree.qza   --o-rooted-tree /content/data/rooted-tree.qza   --p-n-threads 2\n', returncode=0, stdout='Saved FeatureData[AlignedSequence] to: /content/data/aligned-rep-seqs.qza\nSaved FeatureData[AlignedSequence] to: /content/data/masked-aligned-rep-seqs.qza\nSaved Phylogeny[Unrooted] to: /content/data/unrooted-tree.qza\nSaved Phylogeny[Rooted] to: /content/data/rooted-tree.qza\n', stderr='')

In [ ]:
# %%
# Core diversity metrics
# IMPORTANT: Adjust --p-sampling-depth based on feature-table.qzv results!
run_qiime('''
qiime diversity core-metrics-phylogenetic \
  --i-phylogeny /content/data/rooted-tree.qza \
  --i-table /content/data/feature-table.qza \
  --p-sampling-depth 1000 \
  --m-metadata-file /content/data/metadata.tsv \
  --output-dir /content/data/core-metrics-results
''')

Running QIIME 2 command:

qiime diversity core-metrics-phylogenetic   --i-phylogeny /content/data/rooted-tree.qza   --i-table /content/data/feature-table.qza   --p-sampling-depth 1000   --m-metadata-file /content/data/metadata.tsv   --output-dir /content/data/core-metrics-results


Saved FeatureTable[Frequency] to: /content/data/core-metrics-results/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: /content/data/core-metrics-results/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: /content/data/core-metrics-results/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: /content/data/core-metrics-results/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: /content/data/core-metrics-results/evenness_vector.qza
Saved DistanceMatrix to: /content/data/core-metrics-results/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: /content/data/core-metrics-results/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: /content/data/core-metrics-resul

CompletedProcess(args='. /usr/local/miniforge3/etc/profile.d/conda.sh && conda activate qiime2 && \nqiime diversity core-metrics-phylogenetic   --i-phylogeny /content/data/rooted-tree.qza   --i-table /content/data/feature-table.qza   --p-sampling-depth 1000   --m-metadata-file /content/data/metadata.tsv   --output-dir /content/data/core-metrics-results\n', returncode=0, stdout='Saved FeatureTable[Frequency] to: /content/data/core-metrics-results/rarefied_table.qza\nSaved SampleData[AlphaDiversity] to: /content/data/core-metrics-results/faith_pd_vector.qza\nSaved SampleData[AlphaDiversity] to: /content/data/core-metrics-results/observed_features_vector.qza\nSaved SampleData[AlphaDiversity] to: /content/data/core-metrics-results/shannon_vector.qza\nSaved SampleData[AlphaDiversity] to: /content/data/core-metrics-results/evenness_vector.qza\nSaved DistanceMatrix to: /content/data/core-metrics-results/unweighted_unifrac_distance_matrix.qza\nSaved DistanceMatrix to: /content/data/core-metric

# ## 6. Statistical Tests (Fly vs Manure Comparison)

In [ ]:
# %% [markdown]
# ## 6. Statistical Tests (Fly vs Manure Comparison)

# %%
# Alpha diversity significance
run_qiime('''
qiime diversity alpha-group-significance \
  --i-alpha-diversity /content/data/core-metrics-results/shannon_vector.qza \
  --m-metadata-file /content/data/metadata.tsv \
  --o-visualization /content/data/shannon-significance.qzv
''')

run_qiime('''
qiime diversity alpha-group-significance \
  --i-alpha-diversity /content/data/core-metrics-results/observed_features_vector.qza \
  --m-metadata-file /content/data/metadata.tsv \
  --o-visualization /content/data/observed-features-significance.qzv
''')


Running QIIME 2 command:

qiime diversity alpha-group-significance   --i-alpha-diversity /content/data/core-metrics-results/shannon_vector.qza   --m-metadata-file /content/data/metadata.tsv   --o-visualization /content/data/shannon-significance.qzv


Saved Visualization to: /content/data/shannon-significance.qzv

Running QIIME 2 command:

qiime diversity alpha-group-significance   --i-alpha-diversity /content/data/core-metrics-results/observed_features_vector.qza   --m-metadata-file /content/data/metadata.tsv   --o-visualization /content/data/observed-features-significance.qzv


Saved Visualization to: /content/data/observed-features-significance.qzv



CompletedProcess(args='. /usr/local/miniforge3/etc/profile.d/conda.sh && conda activate qiime2 && \nqiime diversity alpha-group-significance   --i-alpha-diversity /content/data/core-metrics-results/observed_features_vector.qza   --m-metadata-file /content/data/metadata.tsv   --o-visualization /content/data/observed-features-significance.qzv\n', returncode=0, stdout='Saved Visualization to: /content/data/observed-features-significance.qzv\n', stderr='')

In [ ]:
# %%
# Beta diversity PERMANOVA (sample_type: fly vs manure)
run_qiime('''
qiime diversity beta-group-significance \
  --i-distance-matrix /content/data/core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file /content/data/metadata.tsv \
  --m-metadata-column sample_type \
  --o-visualization /content/data/unweighted-unifrac-sample-type-significance.qzv \
  --p-pairwise
''')

run_qiime('''
qiime diversity beta-group-significance \
  --i-distance-matrix /content/data/core-metrics-results/weighted_unifrac_distance_matrix.qza \
  --m-metadata-file /content/data/metadata.tsv \
  --m-metadata-column sample_type \
  --o-visualization /content/data/weighted-unifrac-sample-type-significance.qzv \
  --p-pairwise
''')

Running QIIME 2 command:

qiime diversity beta-group-significance   --i-distance-matrix /content/data/core-metrics-results/unweighted_unifrac_distance_matrix.qza   --m-metadata-file /content/data/metadata.tsv   --m-metadata-column sample_type   --o-visualization /content/data/unweighted-unifrac-sample-type-significance.qzv   --p-pairwise


Saved Visualization to: /content/data/unweighted-unifrac-sample-type-significance.qzv

Running QIIME 2 command:

qiime diversity beta-group-significance   --i-distance-matrix /content/data/core-metrics-results/weighted_unifrac_distance_matrix.qza   --m-metadata-file /content/data/metadata.tsv   --m-metadata-column sample_type   --o-visualization /content/data/weighted-unifrac-sample-type-significance.qzv   --p-pairwise


Saved Visualization to: /content/data/weighted-unifrac-sample-type-significance.qzv



CompletedProcess(args='. /usr/local/miniforge3/etc/profile.d/conda.sh && conda activate qiime2 && \nqiime diversity beta-group-significance   --i-distance-matrix /content/data/core-metrics-results/weighted_unifrac_distance_matrix.qza   --m-metadata-file /content/data/metadata.tsv   --m-metadata-column sample_type   --o-visualization /content/data/weighted-unifrac-sample-type-significance.qzv   --p-pairwise\n', returncode=0, stdout='Saved Visualization to: /content/data/weighted-unifrac-sample-type-significance.qzv\n', stderr='')

# ## 7. Download Results

In [ ]:
# %% [markdown]
# ## 7. Download Results

# %%
# Zip all results
!zip -r /content/qiime2_results.zip /content/data/*.qzv /content/data/*.qza /content/data/core-metrics-results/

from google.colab import files
files.download('/content/qiime2_results.zip')

# %% [markdown]
# ## Notes
# - View .qzv files at https://view.qiime2.org
# - Adjust --p-sampling-depth in diversity analysis based on your feature-table.qzv
# - Adjust --p-trunc-len in DADA2 based on single_end_demux.qzv quality plot

  adding: content/data/denoising-stats.qzv (deflated 2%)
  adding: content/data/feature-table.qzv (deflated 4%)
  adding: content/data/observed-features-significance.qzv (deflated 28%)
  adding: content/data/rep-seqs.qzv (deflated 3%)
  adding: content/data/shannon-significance.qzv (deflated 28%)
  adding: content/data/single_end_demux.qzv (deflated 5%)
  adding: content/data/taxa-bar-plots.qzv (deflated 8%)
  adding: content/data/taxonomy.qzv (deflated 3%)
  adding: content/data/unweighted-unifrac-sample-type-significance.qzv (deflated 30%)
  adding: content/data/weighted-unifrac-sample-type-significance.qzv (deflated 30%)
  adding: content/data/aligned-rep-seqs.qza (deflated 7%)
  adding: content/data/denoising-stats.qza (deflated 36%)
  adding: content/data/feature-table.qza (deflated 5%)
  adding: content/data/masked-aligned-rep-seqs.qza (deflated 9%)
  adding: content/data/rep-seqs.qza (deflated 3%)
  adding: content/data/rooted-tree.qza (deflated 23%)
  adding: content/data/singl

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>